In [1]:
import pandas as pd
from pythainlp.tokenize import word_tokenize
from rank_bm25 import BM25Okapi

In [2]:

documents = [
    "วิธีดูกราฟหุ้นเบื้องต้นสำหรับมือใหม่",          # Doc 0
    "แมวน่ารัก พันธุ์สก็อตติชโฟลด์ กินอาหารเม็ด",    # Doc 1
    "การเขียนโปรแกรม Python สำหรับ Data Science", # Doc 2
    "หุ้นไทยวันนี้ ปิดลบ 10 จุด นักลงทุนกังวล",      # Doc 3
    "ร้านอาหารอร่อย แนะนำย่านสยาม",               # Doc 4
    "Python คือภาษาโปรแกรมที่ได้รับความนิยมสูง",    # Doc 5
]

tokenized_corpus = [word_tokenize(doc, engine="newmm") for doc in documents]

#  Indexing (BM25)
# TF-IDF และความยาวประโยคเก็บไว้ใน Index
bm25 = BM25Okapi(tokenized_corpus)

def search(query, n=3):
    tokenized_query = word_tokenize(query, engine="newmm")
    
    doc_scores = bm25.get_scores(tokenized_query)
    
    top_n = bm25.get_top_n(tokenized_query, documents, n=n)
    
    print(f"🔎 ค้นหา: '{query}'")
    print("-" * 30)
    for i, doc in enumerate(top_n):
        print(f"อันดับ {i+1}: {doc}")
    print("\n")

# --- ทดลองใช้งาน ---
search("อยากเขียนโปรแกรม") 
search("หุ้นตก")
search("หาของกิน")

🔎 ค้นหา: 'อยากเขียนโปรแกรม'
------------------------------
อันดับ 1: Python คือภาษาโปรแกรมที่ได้รับความนิยมสูง
อันดับ 2: การเขียนโปรแกรม Python สำหรับ Data Science
อันดับ 3: ร้านอาหารอร่อย แนะนำย่านสยาม


🔎 ค้นหา: 'หุ้นตก'
------------------------------
อันดับ 1: วิธีดูกราฟหุ้นเบื้องต้นสำหรับมือใหม่
อันดับ 2: หุ้นไทยวันนี้ ปิดลบ 10 จุด นักลงทุนกังวล
อันดับ 3: ร้านอาหารอร่อย แนะนำย่านสยาม


🔎 ค้นหา: 'หาของกิน'
------------------------------
อันดับ 1: Python คือภาษาโปรแกรมที่ได้รับความนิยมสูง
อันดับ 2: ร้านอาหารอร่อย แนะนำย่านสยาม
อันดับ 3: หุ้นไทยวันนี้ ปิดลบ 10 จุด นักลงทุนกังวล




In [7]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

print("⏳ กำลังโหลดโมเดล...")
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

documents = [
    "วิธีดูกราฟหุ้นเบื้องต้นสำหรับมือใหม่",
    "แมวน่ารัก พันธุ์สก็อตติชโฟลด์ กินอาหารเม็ด",
    "การเขียนโปรแกรม Python สำหรับ Data Science",
    "หุ้นไทยวันนี้ ปิดลบ 10 จุด นักลงทุนกังวล",
    "ร้านอาหารอร่อย แนะนำย่านสยาม",
    "Python คือภาษาโปรแกรมที่ได้รับความนิยมสูง",
]

corpus_embeddings = model.encode(documents, convert_to_tensor=True)

def semantic_search(query, n=3):
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
    
    top_results = np.argsort(cos_scores.cpu().numpy())[-n:][::-1] # เรียงมากไปน้อย
    
    print(f"🧠 AI ค้นหา: '{query}'")
    print("-" * 40)
    for idx in top_results:
        score = cos_scores[idx]
        print(f"Score: {score:.4f} | เอกสาร: {documents[idx]}")
    print("\n")

semantic_search("สอน Coding หน่อย")  

semantic_search("ตลาดทุนซบเซา") 

⏳ กำลังโหลดโมเดล...


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


🧠 AI ค้นหา: 'สอน Coding หน่อย'
----------------------------------------
Score: 0.4170 | เอกสาร: การเขียนโปรแกรม Python สำหรับ Data Science
Score: 0.3432 | เอกสาร: Python คือภาษาโปรแกรมที่ได้รับความนิยมสูง
Score: 0.1430 | เอกสาร: แมวน่ารัก พันธุ์สก็อตติชโฟลด์ กินอาหารเม็ด


🧠 AI ค้นหา: 'ตลาดทุนซบเซา'
----------------------------------------
Score: 0.5064 | เอกสาร: วิธีดูกราฟหุ้นเบื้องต้นสำหรับมือใหม่
Score: 0.3766 | เอกสาร: ร้านอาหารอร่อย แนะนำย่านสยาม
Score: 0.3490 | เอกสาร: หุ้นไทยวันนี้ ปิดลบ 10 จุด นักลงทุนกังวล


